In [2]:
!nvidia-smi

Fri Jan 17 09:38:21 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-PCIE-16GB           On  | 00000000:21:00.0 Off |                    0 |
| N/A   28C    P0              24W / 250W |      4MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Package Loading & Definition

In [1]:
import MOICVAE.SNF as snf
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchmetrics
from torchmetrics import MeanSquaredError
from sklearn.impute import KNNImputer
import scripts
from functools import lru_cache
import optuna
from sklearn.metrics import pairwise_distances
from sklearn.exceptions import DataConversionWarning
import warnings
from torch.utils.data import Dataset
# 경고 무시 설정
warnings.filterwarnings("ignore", category=DataConversionWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/kim14/.conda/envs/jupyter/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/kim14/.conda/envs/jupyter/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/kim14/.conda/envs/jupyter/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/kim14/.conda/envs/jupyter/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launc

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/kim14/.conda/envs/jupyter/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/kim14/.conda/envs/jupyter/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/kim14/.conda/envs/jupyter/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/kim14/.conda/envs/jupyter/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launc

AttributeError: _ARRAY_API not found

/home/kim14/.conda/envs/jupyter/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/kim14/.conda/envs/jupyter/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/kim14/.conda/envs/jupyter/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)


In [3]:
class OmicsDataset_dict(Dataset): 
    def __init__(self, omic_dict, drug_dict, data): 
        self.omic_dict = omic_dict
        self.drug_dict = drug_dict
        self.cell_mapped_ids = {key:i for i, key in enumerate(self.omic_dict.keys())}
        # omic_dict의 키를 고유한 인덱스로 매핑
        # enumerate는 키들을 순서대로 열거하여 (인덱스, 키) 형태의 튜플로 반환
        # 딕셔너리 컴프레헨션: 각 키를 key로, 각 키의 인덱스를 i로 사용하여 {key:i}형태로 매핑된 딕셔너리 만듬.
        self.drug_mapped_ids = {key:i for i, key in enumerate(self.drug_dict.keys())}
        self.data = data
        
    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, idx): # idx = train_data
        instance = self.data.iloc[idx] 
        cell_id = instance.iloc[0]
        drug_id = instance.iloc[1]
        target = instance.iloc[2]
        
        #omics_data = { # usage of dictionary here causes a problem or crash with collate_fn function in Dataloader 
        #    cell_id : {
        #        data_type: self.omic_dict[cell_id][data_type] for data_type in self.omic_dict[cell_id].keys()
        #    }
        #}
        
        return (torch.cat([self.omic_dict[cell_id][modality] for modality in self.omic_dict[cell_id].keys()]), 
                self.drug_dict[drug_id],
                torch.Tensor([target]),
                torch.Tensor([self.cell_mapped_ids[cell_id]]),
                torch.Tensor([self.drug_mapped_ids[drug_id]])) 

In [4]:
class MultimodalAutoencoder(nn.Module):
    def __init__(self, input_dim, num_modalities = 1,  hidden_dim_encoders = 150, embed_dim = 75, fusion_dim = 150, dropout_encoders = 0.2):
        # get input as a dictionary
        super(MultimodalAutoencoder, self).__init__()
        # EEEEEEEEEEncoder
        self.input_dim = input_dim
        self.num_modalities = num_modalities
        self.do = nn.Dropout(dropout_encoders)

        self.omics_encoder = nn.ModuleList([
            nn.Sequential(
                nn.Linear(input_dim, hidden_dim_encoders), # input 
                nn.ReLU(),
                nn.Linear(hidden_dim_encoders, embed_dim) # encoder hidden layer: 150, 75 as the value from the paper. so we start from this 
            )                                 # I dont get why they used 150, 75 for dimension, but we can tune it later
            for _ in range(num_modalities)
        ])
        # fused latent feature 
        self.fusion_layer = nn.Sequential( # I think we need a fusion layer here, to combine the data modalities
            nn.Linear(embed_dim * num_modalities, fusion_dim),
            nn.ReLU(),
            nn.Linear(fusion_dim, embed_dim) # This concatenate latent features of all omics data, and fusion them and make its dim final latent dim
        )                                     # This is the only way I can think of to fuse omics data
        # decoder
        self.omics_decoder = nn.Sequential(
            nn.Linear(embed_dim, hidden_dim_encoders),
            nn.ReLU(),
            nn.Linear(hidden_dim_encoders, input_dim * num_modalities)
        )
        # I actually dont understand this step in paper. they said that decoder has symmetric structure as encoder,
        # but the data after MDA they provided, has weird dimension(363x90) which makes no sense. this is the point that i cant understand

    def forward(self, x):
        latent_features = [] # get dictionary as an input 
        for i, encoder in enumerate(self.omics_encoder):
            start_idx = i * self.input_dim
            end_idx = start_idx + self.input_dim
            x_modality = x[:, start_idx:end_idx]
            latent_features.append(encoder(self.do(x_modality)))
            
        latent_fused = torch.cat(latent_features, dim=1)
        latent_final = self.fusion_layer(latent_fused)
        decoded = self.omics_decoder(latent_final)
        return decoded, latent_final

In [32]:
# The Model that is modified as pedro said in eamil
class ResNet(nn.Module):
    def __init__(self, embed_dim=256, hidden_dim=1024, dropout=0.1, n_layers=6, norm="layernorm"):
        super().__init__()
        self.mlps = nn.ModuleList()
        
        # Determine normalization layer
        if norm == "layernorm":
            norm_layer = nn.LayerNorm
        elif norm == "batchnorm":
            norm_layer = nn.BatchNorm1d
        else:
            norm_layer = nn.Identity
        
        # Create MLP layers
        for _ in range(n_layers):
            self.mlps.append(
                nn.Sequential(
                    nn.Linear(embed_dim, hidden_dim),
                    norm_layer(hidden_dim),
                    nn.ReLU(),
                    nn.Dropout(dropout),
                    nn.Linear(hidden_dim, embed_dim)
                )
            )
        
        self.lin = nn.Linear(embed_dim, 1)  # Output layer

    def forward(self, x):
        for layer in self.mlps:
            x = (layer(x) + x) / 2  # Residual connection
        return self.lin(x)


class Main_Model(nn.Module):
    def __init__(self, embed_dim=256, hidden_dim=1024, dropout=0.1, n_layers=6, norm="layernorm", 
                 dropout_omics=0.4, dropout_omics_finetuning=0.4):
        super().__init__()
        
        # ResNet as backbone
        self.resnet = ResNet(embed_dim, hidden_dim, dropout, n_layers, norm)
        
        # Modified embed_d: Two-layer MLP with dropout after ReLU
        self.embed_d = nn.Sequential(
            nn.LazyLinear(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, embed_dim)
        )
        
        # Modified embed_c: Two-layer MLP with dropout after ReLU and before first Linear Layer
        self.embed_c = nn.Sequential(
            nn.Dropout(dropout_omics_finetuning),
            nn.LazyLinear(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_omics),
            nn.Linear(hidden_dim, embed_dim)
        )

    def forward(self, c, d):
        # Combine embedded inputs and pass through ResNet
        return self.resnet(self.embed_d(d) + self.embed_c(c))


In [59]:
@lru_cache(maxsize=None)
def get_data_corr(n_fold = 0, fp_radius = 2, transform_into_corr = True, typ = ["rnaseq", "mutations", "cnvs"]):
    # drug
    smile_dict = pd.read_csv("data/smiles.csv", index_col=0)
    fp = scripts.FingerprintFeaturizer(R = fp_radius)
    drug_dict = fp(smile_dict.iloc[:, 1], smile_dict.iloc[:, 0])
    
    # loading all datasets
    driver_genes = pd.read_csv("data/driver_genes.csv").loc[:, "symbol"].dropna()

    rnaseq = pd.read_csv("data/rnaseq_normcount.csv", index_col=0)
    driver_columns = rnaseq.columns.isin(driver_genes)
    filtered_rna = rnaseq.loc[:, driver_columns]
    
    proteomics = pd.read_csv("data/proteomics.csv", index_col=0)
    
    mutation = pd.read_csv("data/binary_mutations.csv")
    mutation.columns = mutation.iloc[0]
    mutation = mutation.iloc[2:,:].set_index("gene_symbol")
    driver_columns = mutation.columns.isin(driver_genes)
    filtered_mut = mutation.loc[:, driver_columns]
    filtered_mut = filtered_mut.astype(float)

    methylations = pd.read_csv("data/methylations.csv",index_col = 0).sort_index(ascending = True)

    cnvs = pd.read_csv("data/copy_number_variations.csv",index_col= 0)

    # concatenate all dataset 
    # inner join based on index: model_ids with NaN are automatically filtered out 
    data_concat = pd.concat([filtered_rna, proteomics, filtered_mut, methylations, cnvs], axis=1, join='inner')
    
    
    # Filter data by common indices in all modalities
    filtered_rna = filtered_rna[filtered_rna.index.isin(data_concat.index)]
    proteomics = proteomics[proteomics.index.isin(data_concat.index)]
    filtered_mut = filtered_mut[filtered_mut.index.isin(data_concat.index)]
    methylations = methylations[methylations.index.isin(data_concat.index)]
    cnvs = cnvs[cnvs.index.isin(data_concat.index)]
    
    # Initialize cell_dict
    cell_dict = {}

    if not transform_into_corr:
        for cell in data_concat.index:
            # Initialize a sub-dictionary for each cell
            concatenated_data = []
            
            # Add data for each type specified in typ
            if "rnaseq" in typ:
                concatenated_data.append(filtered_rna.loc[cell].to_numpy())
            if "proteomics" in typ:
                concatenated_data.append(proteomics.loc[cell].to_numpy())
            if "mutations" in typ:
                concatenated_data.append(filtered_mut.loc[cell].to_numpy())
            if "methylations" in typ:
                concatenated_data.append(methylations.loc[cell].to_numpy())
            if "cnvs" in typ:
                concatenated_data.append(cnvs.loc[cell].to_numpy())

            cell_dict[cell] = torch.Tensor(np.concatenate(concatenated_data))

    # GDSC
    GDSC1 = pd.read_csv("data/GDSC1.csv", index_col=0)
    # default, remove data where lines or drugs are missing:
    data = GDSC1.query("SANGER_MODEL_ID in @data_concat.index & DRUG_ID in @drug_dict.keys()")
    unique_cell_lines = data.loc[:, "SANGER_MODEL_ID"].unique()

    np.random.seed(420) # for comparibility, don't change it!
    np.random.shuffle(unique_cell_lines)
    folds = np.array_split(unique_cell_lines, 10)
    test_lines = folds[0] 
    train_idxs = list(range(10))
    train_idxs.remove(n_fold)
    np.random.seed(420)
    validation_idx = np.random.choice(train_idxs)
    train_idxs.remove(validation_idx)
    train_lines = np.concatenate([folds[idx] for idx in train_idxs])
    validation_lines = folds[validation_idx]
    test_lines = folds[n_fold] 

        # no change needed, query works fine with some missing
    train_data = data.query("SANGER_MODEL_ID in @train_lines")
    validation_data = data.query("SANGER_MODEL_ID in @validation_lines")
    test_data = data.query("SANGER_MODEL_ID in @test_lines")
    
    if transform_into_corr:
        # train, val, test among filtered data
        # these are valid train_, val_ and test_data index
        
        
        n_train = len(train_lines)  
        n_val = len(validation_lines)      
        n_test = len(test_lines)
        
        # Precompute similarity matrices for each data type
        similarity_matrices = {}
        
        if "rnaseq" in typ:
            exp_com = np.corrcoef(np.vstack([filtered_rna.loc[train_lines], 
                                             filtered_rna.loc[validation_lines], 
                                             filtered_rna.loc[test_lines]]), rowvar=True)
            train = exp_com[:n_train, :n_train]
            val = exp_com[n_train:n_train+n_val, :n_train]
            test = exp_com[n_train+n_val:, :n_train]
            sim_combined = np.vstack([train, val, test])
            similarity_matrices["rnaseq"] = sim_combined
        
        if "proteomics" in typ:
            prot_com = np.corrcoef(np.vstack([proteomics.loc[train_lines], 
                                              proteomics.loc[validation_lines], 
                                              proteomics.loc[test_lines]]), rowvar=True)
            train = prot_com[:n_train, :n_train]
            val = prot_com[n_train:n_train+n_val, :n_train]
            test = prot_com[n_train+n_val:, :n_train]
            sim_combined = np.vstack([train, val, test])
            similarity_matrices["proteomics"] = sim_combined
        
        if "mutations" in typ:
            train_snp = filtered_mut.loc[train_lines].astype(bool)
            val_snp = filtered_mut.loc[validation_lines].astype(bool)
            test_snp = filtered_mut.loc[test_lines].astype(bool)
            
            train = 1 - pairwise_distances(train_snp.values, train_snp.values, metric="jaccard")
            val = 1 - pairwise_distances(val_snp.values, train_snp.values, metric="jaccard")
            test = 1 - pairwise_distances(test_snp.values, train_snp.values, metric="jaccard")
    
            sim_combined = np.vstack([train, val, test])
            similarity_matrices["mutations"] = sim_combined
        
        if "methylations" in typ:
            methyl_com = np.corrcoef(np.vstack([methylations.loc[train_lines], 
                                                methylations.loc[validation_lines], 
                                                methylations.loc[test_lines]]), rowvar=True)
            train = methyl_com[:n_train, :n_train]
            val = methyl_com[n_train:n_train+n_val, :n_train]
            test = methyl_com[n_train+n_val:, :n_train]
            sim_combined = np.vstack([train, val, test])
            similarity_matrices["mathylations"] = sim_combined
        
        if "cnvs" in typ:
            cnv_com = np.nan_to_num(np.corrcoef(np.vstack([cnvs.loc[train_lines], # nan-generation problem fixed 
                                             cnvs.loc[validation_lines], 
                                             cnvs.loc[test_lines]]), rowvar=True))
            train= cnv_com[:n_train, :n_train]
            val= cnv_com[n_train:n_train+n_val, :n_train]
            test= cnv_com[n_train+n_val:, :n_train]
            sim_combined = np.vstack([train, val, test])
            similarity_matrices["cnvs"] = sim_combined
            
        cell_dict = {}

        # 
        for cell in unique_cell_lines:
            cell_dict[cell] = {}
            for data_type in typ:
                sim_matrices = similarity_matrices[data_type]
                sim_tensor = torch.Tensor(sim_matrices)
                cell_idx = np.where(unique_cell_lines == cell)[0][0]
                cell_dict[cell][data_type] = sim_tensor[cell_idx]
    
        return (OmicsDataset_dict(cell_dict, drug_dict, train_data),
        OmicsDataset_dict(cell_dict, drug_dict, validation_data),
        OmicsDataset_dict(cell_dict, drug_dict, test_data))

    return (scripts.OmicsDataset(cell_dict, drug_dict, train_data),
    scripts.OmicsDataset(cell_dict, drug_dict, validation_data),
    scripts.OmicsDataset(cell_dict, drug_dict, test_data))
    

In [55]:
# to use data 
n_fold = 0
fp_radius = 2
transform_into_corr = True
typ = ["rnaseq", "mutations", "cnvs"]

# drug
smile_dict = pd.read_csv("data/smiles.csv", index_col=0)
fp = scripts.FingerprintFeaturizer(R = fp_radius)
drug_dict = fp(smile_dict.iloc[:, 1], smile_dict.iloc[:, 0])

# loading all datasets
driver_genes = pd.read_csv("data/driver_genes.csv").loc[:, "symbol"].dropna()

rnaseq = pd.read_csv("data/rnaseq_normcount.csv", index_col=0)
driver_columns = rnaseq.columns.isin(driver_genes)
filtered_rna = rnaseq.loc[:, driver_columns]

proteomics = pd.read_csv("data/proteomics.csv", index_col=0)

mutation = pd.read_csv("data/binary_mutations.csv")
mutation.columns = mutation.iloc[0]
mutation = mutation.iloc[2:,:].set_index("gene_symbol")
driver_columns = mutation.columns.isin(driver_genes)
filtered_mut = mutation.loc[:, driver_columns]
filtered_mut = filtered_mut.astype(float)

methylations = pd.read_csv("data/methylations.csv",index_col = 0).sort_index(ascending = True)

cnvs = pd.read_csv("data/copy_number_variations.csv",index_col= 0)

# concatenate all dataset 
# inner join based on index: model_ids with NaN are automatically filtered out 
data_concat = pd.concat([filtered_rna, proteomics, filtered_mut, methylations, cnvs], axis=1, join='inner')


# Filter data by common indices in all modalities
filtered_rna = filtered_rna[filtered_rna.index.isin(data_concat.index)]
proteomics = proteomics[proteomics.index.isin(data_concat.index)]
filtered_mut = filtered_mut[filtered_mut.index.isin(data_concat.index)]
methylations = methylations[methylations.index.isin(data_concat.index)]
cnvs = cnvs[cnvs.index.isin(data_concat.index)]

# Initialize cell_dict
cell_dict = {}

if not transform_into_corr:
    for cell in data_concat.index:
        # Initialize a sub-dictionary for each cell
        concatenated_data = []
        
        # Add data for each type specified in typ
        if "rnaseq" in typ:
            concatenated_data.append(filtered_rna.loc[cell].to_numpy())
        if "proteomics" in typ:
            concatenated_data.append(proteomics.loc[cell].to_numpy())
        if "mutations" in typ:
            concatenated_data.append(filtered_mut.loc[cell].to_numpy())
        if "methylations" in typ:
            concatenated_data.append(methylations.loc[cell].to_numpy())
        if "cnvs" in typ:
            concatenated_data.append(cnvs.loc[cell].to_numpy())

# GDSC
GDSC1 = pd.read_csv("data/GDSC1.csv", index_col=0)
# default, remove data where lines or drugs are missing:
data = GDSC1.query("SANGER_MODEL_ID in @data_concat.index & DRUG_ID in @drug_dict.keys()")
unique_cell_lines = data.loc[:, "SANGER_MODEL_ID"].unique()

np.random.seed(420) # for comparibility, don't change it!
np.random.shuffle(unique_cell_lines)
folds = np.array_split(unique_cell_lines, 10)
test_lines = folds[0] 
train_idxs = list(range(10))
train_idxs.remove(n_fold)
np.random.seed(420)
validation_idx = np.random.choice(train_idxs)
train_idxs.remove(validation_idx)
train_lines = np.concatenate([folds[idx] for idx in train_idxs])
validation_lines = folds[validation_idx]
test_lines = folds[n_fold] 

    # no change needed, query works fine with some missing
train_data = data.query("SANGER_MODEL_ID in @train_lines")
validation_data = data.query("SANGER_MODEL_ID in @validation_lines")
test_data = data.query("SANGER_MODEL_ID in @test_lines")



[12:01:47] DEPRECATION WARNING: please use MorganGenerator
[12:01:47] DEPRECATION WARNING: please use MorganGenerator
[12:01:47] DEPRECATION WARNING: please use MorganGenerator
[12:01:47] DEPRECATION WARNING: please use MorganGenerator
[12:01:47] DEPRECATION WARNING: please use MorganGenerator
[12:01:47] DEPRECATION WARNING: please use MorganGenerator
[12:01:47] DEPRECATION WARNING: please use MorganGenerator
[12:01:47] DEPRECATION WARNING: please use MorganGenerator
[12:01:47] DEPRECATION WARNING: please use MorganGenerator
[12:01:47] DEPRECATION WARNING: please use MorganGenerator
[12:01:47] DEPRECATION WARNING: please use MorganGenerator
[12:01:48] DEPRECATION WARNING: please use MorganGenerator
[12:01:48] DEPRECATION WARNING: please use MorganGenerator
[12:01:48] DEPRECATION WARNING: please use MorganGenerator
[12:01:48] DEPRECATION WARNING: please use MorganGenerator
[12:01:48] DEPRECATION WARNING: please use MorganGenerator
[12:01:48] DEPRECATION WARNING: please use MorganGenerat

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [39]:
transform_into_corr = False
if transform_into_corr:
    # train, val, test among filtered data
    # these are valid train_, val_ and test_data index
    
    
    n_train = len(train_lines)  
    n_val = len(validation_lines)      
    n_test = len(test_lines)
    
    # Precompute similarity matrices for each data type
    similarity_matrices = {}
    
    if "rnaseq" in typ:
        exp_com = np.corrcoef(np.vstack([filtered_rna.loc[train_lines], 
                                         filtered_rna.loc[validation_lines], 
                                         filtered_rna.loc[test_lines]]), rowvar=True)
        train = exp_com[:n_train, :n_train]
        val = exp_com[n_train:n_train+n_val, :n_train]
        test = exp_com[n_train+n_val:, :n_train]
        sim_combined = np.vstack([train, val, test])
        similarity_matrices["rnaseq"] = sim_combined
    
    if "proteomics" in typ:
        prot_com = np.corrcoef(np.vstack([proteomics.loc[train_lines], 
                                          proteomics.loc[validation_lines], 
                                          proteomics.loc[test_lines]]), rowvar=True)
        train = prot_com[:n_train, :n_train]
        val = prot_com[n_train:n_train+n_val, :n_train]
        test = prot_com[n_train+n_val:, :n_train]
        sim_combined = np.vstack([train, val, test])
        similarity_matrices["proteomics"] = sim_combined
    
    if "mutations" in typ:
        train_snp = filtered_mut.loc[train_lines].astype(bool)
        val_snp = filtered_mut.loc[validation_lines].astype(bool)
        test_snp = filtered_mut.loc[test_lines].astype(bool)
        
        train = 1 - pairwise_distances(train_snp.values, train_snp.values, metric="jaccard")
        val = 1 - pairwise_distances(val_snp.values, train_snp.values, metric="jaccard")
        test = 1 - pairwise_distances(test_snp.values, train_snp.values, metric="jaccard")

        sim_combined = np.vstack([train, val, test])
        similarity_matrices["mutations"] = sim_combined
    
    if "methylations" in typ:
        methyl_com = np.corrcoef(np.vstack([methylations.loc[train_lines], 
                                            methylations.loc[validation_lines], 
                                            methylations.loc[test_lines]]), rowvar=True)
        train = methyl_com[:n_train, :n_train]
        val = methyl_com[n_train:n_train+n_val, :n_train]
        test = methyl_com[n_train+n_val:, :n_train]
        sim_combined = np.vstack([train, val, test])
        similarity_matrices["mathylations"] = sim_combined
    
    if "cnvs" in typ:
        cnv_com = np.nan_to_num(np.corrcoef(np.vstack([cnvs.loc[train_lines], # nan-generation problem fixed 
                                         cnvs.loc[validation_lines], 
                                         cnvs.loc[test_lines]]), rowvar=True))
        train= cnv_com[:n_train, :n_train]
        val= cnv_com[n_train:n_train+n_val, :n_train]
        test= cnv_com[n_train+n_val:, :n_train]
        sim_combined = np.vstack([train, val, test])
        similarity_matrices["cnvs"] = sim_combined
        
    cell_dict = {}

    # 
    for cell in unique_cell_lines:
            cell_dict[cell] = {}
            for data_type in typ:
                sim_matrices = similarity_matrices[data_type]
                sim_tensor = torch.Tensor(sim_matrices)
                cell_idx = np.where(unique_cell_lines == cell)[0][0]
                cell_dict[cell][data_type] = sim_tensor[cell_idx]

In [13]:
config = {"features" : {"fp_radius":2,
                        "use_correlation_representation": True,
                        "num_modalities": 3},
          "optimizer": {"batch_size": 220,
                        "clip_norm":19,
                        "learning_rate": 0.0004592646200179472,
                        "stopping_patience":15,
                        "pre_batch_size": 200,
                        "lr_pretraining": 0.0004592646200179472},
          "model":{"embed_dim":485, # shared
                 "hidden_dim":696, 
                 "dropout":0.48541242824674574, 
                 "n_layers": 4, 
                 "norm": "batchnorm", 
                 "hidden_dim_encoders": 696, # ENCODER
                 "fusion_dim": 700, # ENCODER
                 "dropout_encoders": 0.2,
                 "dropout_omics": 0.4, # second
                 "dropout_omics_finetuning": 0.4, # first
                 "pre_training_epochs": 100}, 
         "env": {"fold": 0,  
                "device":"cuda:0", 
                 "max_epochs": 100, 
                 "search_hyperparameters":False}} 

# Pre-training

In [8]:
pre_train_dataset, pre_val_dataset, pre_test_dataset = get_data_corr(n_fold = config["env"]["fold"],
                                                           fp_radius = config["features"]["fp_radius"], 
                                                           transform_into_corr = config["features"]["use_correlation_representation"],
                                                           typ = ("rnaseq", "mutations", "cnvs"))

[09:39:32] DEPRECATION WARNING: please use MorganGenerator
[09:39:32] DEPRECATION WARNING: please use MorganGenerator
[09:39:32] DEPRECATION WARNING: please use MorganGenerator
[09:39:32] DEPRECATION WARNING: please use MorganGenerator
[09:39:32] DEPRECATION WARNING: please use MorganGenerator
[09:39:32] DEPRECATION WARNING: please use MorganGenerator
[09:39:32] DEPRECATION WARNING: please use MorganGenerator
[09:39:32] DEPRECATION WARNING: please use MorganGenerator
[09:39:32] DEPRECATION WARNING: please use MorganGenerator
[09:39:32] DEPRECATION WARNING: please use MorganGenerator
[09:39:32] DEPRECATION WARNING: please use MorganGenerator
[09:39:32] DEPRECATION WARNING: please use MorganGenerator
[09:39:32] DEPRECATION WARNING: please use MorganGenerator
[09:39:32] DEPRECATION WARNING: please use MorganGenerator
[09:39:32] DEPRECATION WARNING: please use MorganGenerator
[09:39:32] DEPRECATION WARNING: please use MorganGenerator
[09:39:32] DEPRECATION WARNING: please use MorganGenerat

In [15]:
pre_train_loader = torch.utils.data.DataLoader(
        pre_train_dataset,
        batch_size = config["optimizer"]["pre_batch_size"],
        shuffle=True,
        drop_last=True
    )

#if validation_dataset is not None:
#    val_loader = torch.utils.data.DataLoader(val_dataset,batch_size = batch_size,shuffle=True,drop_last=True)
    
sample_batch = next(iter(train_loader))
sample_omics_data = sample_batch[0]  # 첫 번째 요소는 omics_data (dict)
mod_len = sample_omics_data.shape[1] // config["features"]["num_modalities"]
#input_dict = {f"modality_{i+1}": mod_len for i in range(num_modality)}

pre_model = MultimodalAutoencoder(input_dim = mod_len, 
                              hidden_dim_encoders = config["model"]["hidden_dim_encoders"],
                              embed_dim = config["model"]["embed_dim"],
                              fusion_dim = config["model"]["fusion_dim"],
                              num_modalities = config["features"]["num_modalities"], 
                              dropout_encoders = config["model"]["dropout_encoders"])

optimizer = torch.optim.Adam(model.parameters(), lr=config["optimizer"]["lr_pretraining"])
criterion = nn.MSELoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pre_model.to(device)

# model training
for epoch in range(config["model"]["pre_training_epochs"]):
    total_loss = []
    pre_model.train()


#    if validation_dataset is not None:
#        validation_metrics = evaluate_step(model,val_loader, metrics, device)
#        if epoch > 0 & use_momentum:
#            val_target = 0.2*val_target + 0.8*validation_metrics['R_cellwise_residuals']
#        else:
#            val_target = validation_metrics['R_cellwise_residuals']

    
    # train step
    for batch in pre_train_loader:
        inputs = batch[0].to(device)

        optimizer.zero_grad() # gradient initialization
        reconstructed, latent = pre_model(inputs)

        #target_data = torch.cat([inputs[modality] for modality in inputs], dim=1)
        # I think, here inappropriate loss is used. 
        # From what I know, Autoencoder reconstructs input feature, and calculate loss through comparing input feature and reconstructed feature.
        # So, calculating MSE comparing with target_data, which is drug sensitivity, is nonsense. 
        #loss = criterion(reconstructed, target_data)
        
        # here is a new loss function, but it still outputs nan value as a loss...
        loss = criterion(reconstructed, inputs)
        
        loss.backward()
        #torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=config["optimizer"]["clip_norm"])
        optimizer.step()

        total_loss.append(loss.item())
        
    train_loss = np.mean(total_loss)
    scheduler.step(train_loss)

    print(f'Epoch {epoch + 1}/{config["model"]["pre_training_epochs"]}, Loss: {train_loss:.5f}')

torch.save(pre_model.fusion_layer.state_dict(), "trained_models/pretrained_omics.pth") # save pre-trained weights from fusion layer
print("Training complete!")


Epoch 1/100, Loss: 0.00216
Epoch 2/100, Loss: 0.00053
Epoch 3/100, Loss: 0.00029
Epoch 4/100, Loss: 0.00018
Epoch 5/100, Loss: 0.00014
Epoch 6/100, Loss: 0.00011
Epoch 7/100, Loss: 0.00009
Epoch 8/100, Loss: 0.00007
Epoch 9/100, Loss: 0.00007
Epoch 10/100, Loss: 0.00006
Epoch 11/100, Loss: 0.00005
Epoch 12/100, Loss: 0.00005
Epoch 13/100, Loss: 0.00004
Epoch 14/100, Loss: 0.00004
Epoch 15/100, Loss: 0.00004
Epoch 16/100, Loss: 0.00004
Epoch 17/100, Loss: 0.00003
Epoch 18/100, Loss: 0.00003
Epoch 19/100, Loss: 0.00003
Epoch 20/100, Loss: 0.00003
Epoch 21/100, Loss: 0.00003
Epoch 22/100, Loss: 0.00003
Epoch 23/100, Loss: 0.00003
Epoch 24/100, Loss: 0.00003
Epoch 25/100, Loss: 0.00002
Epoch 26/100, Loss: 0.00002
Epoch 27/100, Loss: 0.00002
Epoch 28/100, Loss: 0.00002
Epoch 29/100, Loss: 0.00002
Epoch 30/100, Loss: 0.00002
Epoch 31/100, Loss: 0.00002
Epoch 32/100, Loss: 0.00002
Epoch 33/100, Loss: 0.00002
Epoch 34/100, Loss: 0.00002
Epoch 35/100, Loss: 0.00002
Epoch 36/100, Loss: 0.00002
E

In [19]:
# evaluation MDA

pre_eval_loader = torch.utils.data.DataLoader(
        pre_test_dataset,
        batch_size = config["optimizer"]["pre_batch_size"],
        shuffle=False, # we dont need this in evaluation step => repeatable result, for comparision
        drop_last=False # to use all data
    )

pre_model.to(device)
pre_model.eval()

eval_losses = []

for x in pre_eval_loader: 
    with torch.no_grad(): # no autograd => in evaluation step we dont need to update weights
        inputs = batch[0].to(device)
        reconstructed, _ = pre_model(inputs)
        evel_loss = criterion(reconstructed,inputs)
        eval_losses.append(evel_loss.item())

final_eval_loss = np.mean(eval_losses)
print(f"Final MSE of Model: {final_eval_loss:.5f}")
        

Final MSE of Model: 0.00000


# Model Training & Evaluation

In [36]:
# 이제 계획에 따라, embed_c가 저 latent feature를 받아야함

def train_step(model, optimizer, loader, config, device):
    loss = nn.MSELoss()
    ls = []
    model.train()
    for x in loader:
        optimizer.zero_grad()
        out = model(x[0].to(device), x[1].to(device))
        l = loss(out.squeeze(), x[2].to(device).squeeze())
        l.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), config["optimizer"]["clip_norm"])
        ls += [l.item()]
        optimizer.step()
    return np.mean(ls)

def finetune_model_train(config, train_dataset, validation_dataset=None, use_momentum=True, callback_epoch = None):
    train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=config["optimizer"]["batch_size"],
                                           drop_last=True,
                                          shuffle=True)
    if validation_dataset is not None:
        val_loader = torch.utils.data.DataLoader(validation_dataset,
                                               batch_size=config["optimizer"]["batch_size"],
                                               drop_last=False,
                                              shuffle=False)
        
    model = Main_Model(embed_dim=config["model"]["embed_dim"],
                       hidden_dim=config["model"]["hidden_dim"], 
                       dropout=config["model"]["dropout"], 
                       n_layers=config["model"]["n_layers"],  
                       dropout_omics=config["model"]["dropout_omics"], 
                       dropout_omics_finetuning=config["model"]["dropout_omics_finetuning"],
                       norm=config["model"]["norm"])
    
    model.embed_c.load_state_dict(torch.load("trained_models/pretrained_omics.pth"), strict=False) # pre-trained weights
    optimizer = torch.optim.Adam(model.parameters(), config["optimizer"]["learning_rate"])
    device = torch.device(config["env"]["device"])
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=5)
    early_stop = scripts.EarlyStop(config["optimizer"]["stopping_patience"])
    model.to(device)
    metrics = torchmetrics.MetricTracker(torchmetrics.MetricCollection(
    {"R_cellwise_residuals":scripts.GroupwiseMetric(metric=torchmetrics.functional.pearson_corrcoef,
                          grouping="drugs",
                          average="macro",
                          residualize=True),
    "R_cellwise":scripts.GroupwiseMetric(metric=torchmetrics.functional.pearson_corrcoef,
                          grouping="cell_lines",
                          average="macro",
                          residualize=False),
    "MSE":torchmetrics.MeanSquaredError()}))
    metrics.to(device)
    use_momentum = True    
    for epoch in range(config["env"]["max_epochs"]):
        train_loss = train_step(model, optimizer, train_loader, config, device)
        lr_scheduler.step(train_loss)
        if validation_dataset is not None:
            validation_metrics = evaluate_step(model,val_loader, metrics, device)
            if epoch > 0 & use_momentum:
                val_target = 0.2*val_target + 0.8*validation_metrics['R_cellwise_residuals']
            else:
                val_target = validation_metrics['R_cellwise_residuals']
        else:
            val_target = None
        if callback_epoch is None:
            print(f"epoch : {epoch}: train loss: {train_loss} Smoothed R interaction (validation) {val_target}")
        else:
            callback_epoch(epoch, val_target)
        if early_stop(train_loss):
            break
    return val_target, model

In [60]:
fine_train_dataset, fine_val_dataset, fine_test_dataset = get_data_corr(n_fold = config["env"]["fold"],
                                                           fp_radius = config["features"]["fp_radius"], 
                                                           transform_into_corr = False,
                                                           typ = ("rnaseq", "mutations", "cnvs"))

[12:04:05] DEPRECATION WARNING: please use MorganGenerator
[12:04:05] DEPRECATION WARNING: please use MorganGenerator
[12:04:05] DEPRECATION WARNING: please use MorganGenerator
[12:04:05] DEPRECATION WARNING: please use MorganGenerator
[12:04:05] DEPRECATION WARNING: please use MorganGenerator
[12:04:05] DEPRECATION WARNING: please use MorganGenerator
[12:04:05] DEPRECATION WARNING: please use MorganGenerator
[12:04:05] DEPRECATION WARNING: please use MorganGenerator
[12:04:05] DEPRECATION WARNING: please use MorganGenerator
[12:04:05] DEPRECATION WARNING: please use MorganGenerator
[12:04:05] DEPRECATION WARNING: please use MorganGenerator
[12:04:05] DEPRECATION WARNING: please use MorganGenerator
[12:04:05] DEPRECATION WARNING: please use MorganGenerator
[12:04:05] DEPRECATION WARNING: please use MorganGenerator
[12:04:05] DEPRECATION WARNING: please use MorganGenerator
[12:04:05] DEPRECATION WARNING: please use MorganGenerator
[12:04:05] DEPRECATION WARNING: please use MorganGenerat

In [61]:
_, main_model = finetune_model_train(config, torch.utils.data.ConcatDataset([fine_train_dataset, fine_val_dataset]), None, use_momentum=False)
device = torch.device(config["env"]["device"])
metrics = torchmetrics.MetricTracker(torchmetrics.MetricCollection(
    {"R_cellwise_residuals":scripts.GroupwiseMetric(metric=torchmetrics.functional.pearson_corrcoef,
                          grouping="drugs",
                          average="macro",
                          residualize=True),
    "R_cellwise":scripts.GroupwiseMetric(metric=torchmetrics.functional.pearson_corrcoef,
                          grouping="cell_lines",
                          average="macro",
                          residualize=False),
    "MSE":torchmetrics.MeanSquaredError()}))
metrics.to(device)
fine_test_dataloader = torch.utils.data.DataLoader(fine_test_dataset,
                                       batch_size=config["optimizer"]["batch_size"],
                                       drop_last=False,
                                      shuffle=False,
                                      pin_memory=True)

/home/kim14/.conda/envs/jupyter/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
/home/kim14/.conda/envs/jupyter/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: FutureWarning: The default value for `maximize` will be changed from `True` to `None` in v1.7.0 of TorchMetrics,will automatically infer the value based on the `higher_is_better` attribute of the metric (if such attribute exists) or raise an error if it does not. If you are explicitly setting the `maximize` argument to either `True` or `False` already, you can ignore this warning.
  warnings.warn(*args, **kwargs)  # noqa: B028


epoch : 0: train loss: 2.101567306830945 Smoothed R interaction (validation) None
epoch : 1: train loss: 1.6592423534216485 Smoothed R interaction (validation) None
epoch : 2: train loss: 1.5798197993536667 Smoothed R interaction (validation) None
epoch : 3: train loss: 1.5325949221517896 Smoothed R interaction (validation) None
epoch : 4: train loss: 1.5012005019394223 Smoothed R interaction (validation) None
epoch : 5: train loss: 1.4624819291389475 Smoothed R interaction (validation) None
epoch : 6: train loss: 1.4385447586127942 Smoothed R interaction (validation) None
epoch : 7: train loss: 1.4035751995698482 Smoothed R interaction (validation) None
epoch : 8: train loss: 1.3859120936712022 Smoothed R interaction (validation) None
epoch : 9: train loss: 1.3628191977407789 Smoothed R interaction (validation) None
epoch : 10: train loss: 1.3396166744279332 Smoothed R interaction (validation) None
epoch : 11: train loss: 1.3202774819396483 Smoothed R interaction (validation) None
epo

In [62]:
main_final_metrics_nohyper = scripts.evaluate_step(main_model, fine_test_dataloader, metrics, device, save_predictions = True, model_name = "pretrained_NoOptHyper", dataset_name = "sim_matrix")
print(f"main model final metrics: {main_final_metrics_nohyper}")

/work/kim14/project_work/scripts/models.py:69: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525552843/work/aten/src/ATen/native/TensorShape.cpp:3277.)
  return torch.linalg.solve(A, Xy).T


Predictions saved to: results/pred_pretrained_NoOptHyper_sim_matrix_20250117_12:39:33_26ac88f3-9881-4cde-9d43-1aa1e1a82008.csv
main model final metrics: {'MSE': 1.5677202939987183, 'R_cellwise': 0.8969780206680298, 'R_cellwise_residuals': 0.3421277403831482}


huh it got best MSE

# Narrow-sense Hyperparameter Optimization

In [ ]:
# config for hyperparameter estimation

config["model"] = {"embed_dim":trial.suggest_int("embed_dim", 64, 512),
                 "hidden_dim":696, # hidden layer의 차원
                 "dropout":0.48541242824674574, # 40퍼센트의 노드를 랜덤하게 드랍아웃 
                 "n_layers": 4, # 3개의 hidden layer를 사용
                 "norm": "batchnorm", # batch normalization을 사용하여 모델이 학습 중 출력 분포를 정규화하여 학습을 안정화
                 "hidden_dim_encoders": trial.suggest_int("hidden_dim_encoders", 64, 2048),
                 "fusion_dim": trial.suggest_int("fusion_dim", 64, 2048),
                 "dropout_encoders": trial.suggest_float("dropout_omics", 0.0, 0.5),
                 "dropout_omics": trial.suggest_float("dropout_omics", 0.0, 0.9), # what is this for?
                 "dropout_omics_finetuning": trial.suggest_float("dropout_omics_finetuning", 0.0, 0.9),
                 "pre_training_epochs": trial.suggest_int("pre_training_epochs", 1, 500)}

config["optimizer"] = {"batch_size": 220,
                        "clip_norm":19,
                        "learning_rate": 0.0004592646200179472,
                        "stopping_patience":15,
                        "pre_batch_size": trial.suggest_int("pre_batch_size", 128, 512),
                        "lr_pretraining": trial.suggest_float("lr_pretraining", 1e-6, 1e-1, log=True)}

# Mülleimer

In [3]:
@lru_cache(maxsize=None)
def get_data_corr_with_filtering(n_fold = 0, fp_radius = 2, transform_into_corr = True, typ = ["rnaseq", "mutations", "cnvs"]):
    # drug
    smile_dict = pd.read_csv("data/smiles.csv", index_col=0)
    fp = scripts.FingerprintFeaturizer(R = fp_radius)
    drug_dict = fp(smile_dict.iloc[:, 1], smile_dict.iloc[:, 0])
    
    # loading all datasets
    driver_genes = pd.read_csv("data/driver_genes.csv").loc[:, "symbol"].dropna()

    rnaseq = pd.read_csv("data/rnaseq_normcount.csv", index_col=0)
    driver_columns = rnaseq.columns.isin(driver_genes)
    filtered_rna = rnaseq.loc[:, driver_columns]
    
    proteomics = pd.read_csv("data/proteomics.csv", index_col=0)
    
    mutation = pd.read_csv("data/binary_mutations.csv")
    mutation.columns = mutation.iloc[0]
    mutation = mutation.iloc[2:,:].set_index("gene_symbol")
    driver_columns = mutation.columns.isin(driver_genes)
    filtered_mut = mutation.loc[:, driver_columns]
    filtered_mut = filtered_mut.astype(float)

    methylations = pd.read_csv("data/methylations.csv",index_col = 0).sort_index(ascending = True)

    cnvs = pd.read_csv("data/copy_number_variations.csv",index_col= 0)

    # concatenate all dataset 
    # inner join based on index: model_ids with NaN are automatically filtered out 
    data_concat = pd.concat([filtered_rna, proteomics, filtered_mut, methylations, cnvs], axis=1, join='inner')
    
    
    # Filter data by common indices in all modalities
    filtered_rna = filtered_rna[filtered_rna.index.isin(data_concat.index)]
    proteomics = proteomics[proteomics.index.isin(data_concat.index)]
    filtered_mut = filtered_mut[filtered_mut.index.isin(data_concat.index)]
    methylations = methylations[methylations.index.isin(data_concat.index)]
    cnvs = cnvs[cnvs.index.isin(data_concat.index)]
    
    # Initialize cell_dict
    cell_dict = {}

    if not transform_into_corr:
        for cell in data_concat.index:
            # Initialize a sub-dictionary for each cell
            cell_dict[cell] = {}
            
            # Add data for each type specified in typ
            if "rnaseq" in typ:
                cell_dict[cell]["rnaseq"] = torch.Tensor(filtered_rna.loc[cell].to_numpy())
            if "proteomics" in typ:
                cell_dict[cell]["proteomics"] = torch.Tensor(proteomics.loc[cell].to_numpy())
            if "mutations" in typ:
                cell_dict[cell]["mutations"] = torch.Tensor(filtered_mut.loc[cell].to_numpy())
            if "methylations" in typ:
                cell_dict[cell]["methylations"] = torch.Tensor(methylations.loc[cell].to_numpy())
            if "cnvs" in typ:
                cell_dict[cell]["cnvs"] = torch.Tensor(cnvs.loc[cell].to_numpy())

    # GDSC
    GDSC1 = pd.read_csv("data/GDSC1.csv", index_col=0)
    # default, remove data where lines or drugs are missing:
    data = GDSC1.query("SANGER_MODEL_ID in @data_concat.index & DRUG_ID in @drug_dict.keys()")
    unique_cell_lines = data.loc[:, "SANGER_MODEL_ID"].unique()

    np.random.seed(420) # for comparibility, don't change it!
    np.random.shuffle(unique_cell_lines)
    folds = np.array_split(unique_cell_lines, 10)
    test_lines = folds[0] 
    train_idxs = list(range(10))
    train_idxs.remove(n_fold)
    np.random.seed(420)
    validation_idx = np.random.choice(train_idxs)
    train_idxs.remove(validation_idx)
    train_lines = np.concatenate([folds[idx] for idx in train_idxs])
    validation_lines = folds[validation_idx]
    test_lines = folds[n_fold] 
    
    if transform_into_corr:
        # ic50 filtering
        ic50_mat = data.pivot(index = 'SANGER_MODEL_ID', columns = 'DRUG_ID', values = 'LN_IC50')
        drug_nan_ratio = ic50_mat.isna().mean(axis=0) 
        cellline_nan_ratio = ic50_mat.isna().mean(axis=1)
        filtered_ic50 = ic50_mat.loc[cellline_nan_ratio < 0.3, drug_nan_ratio < 0.3]
        imputer = KNNImputer(n_neighbors=5)  # k-NN에서 k=5
        imputed_ic50 = pd.DataFrame(
            imputer.fit_transform(filtered_ic50),
            index=filtered_ic50.index,
            columns=filtered_ic50.columns)      
        t = imputed_ic50.median()
        binarized_ic50 = imputed_ic50.apply(lambda x: x.apply(lambda v: 1 if v <= t[x.name] else 0), axis=0)
        
        # index filtering, here only exp, mutation, cnv data are used
        cell_line_index = binarized_ic50.index.intersection(data_concat.index)
        ic50 = binarized_ic50.loc[cell_line_index]
        
        # train, val, test among filtered data
        # these are valid train_, val_ and test_data index
        train_lines = np.intersect1d(train_lines, ic50.index)
        valid_validation_lines = np.intersect1d(validation_lines, ic50.index)
        valid_test_lines = np.intersect1d(test_lines, ic50.index)
        
        n_train = len(train_lines)  
        n_val = len(valid_validation_lines)      
        n_test = len(valid_test_lines)
        
        # Precompute similarity matrices for each data type
        similarity_matrices = {}
        
        if "rnaseq" in typ:
            exp_com = np.corrcoef(np.vstack([filtered_rna.loc[train_lines], 
                                             filtered_rna.loc[valid_validation_lines], 
                                             filtered_rna.loc[valid_test_lines]]), rowvar=True)
            train = exp_com[:n_train, :n_train]
            val = exp_com[n_train:n_train+n_val, :n_train]
            test = exp_com[n_train+n_val:, :n_train]
            sim_combined = np.vstack([train, val, test])
            similarity_matrices["rnaseq"] = sim_combined
        
        if "proteomics" in typ:
            prot_com = np.corrcoef(np.vstack([proteomics.loc[train_lines], 
                                              proteomics.loc[valid_validation_lines], 
                                              proteomics.loc[valid_test_lines]]), rowvar=True)
            train = prot_com[:n_train, :n_train]
            val = prot_com[n_train:n_train+n_val, :n_train]
            test = prot_com[n_train+n_val:, :n_train]
            sim_combined = np.vstack([train, val, test])
            similarity_matrices["proteomics"] = sim_combined
        
        if "mutations" in typ:
            train_snp = filtered_mut.loc[train_lines].astype(bool)
            val_snp = filtered_mut.loc[valid_validation_lines].astype(bool)
            test_snp = filtered_mut.loc[valid_test_lines].astype(bool)
            
            train = 1 - pairwise_distances(train_snp.values, train_snp.values, metric="jaccard")
            val = 1 - pairwise_distances(val_snp.values, train_snp.values, metric="jaccard")
            test = 1 - pairwise_distances(test_snp.values, train_snp.values, metric="jaccard")
    
            sim_combined = np.vstack([train, val, test])
            similarity_matrices["mutations"] = sim_combined
        
        if "methylations" in typ:
            methyl_com = np.corrcoef(np.vstack([methylations.loc[train_lines], 
                                                methylations.loc[valid_validation_lines], 
                                                methylations.loc[valid_test_lines]]), rowvar=True)
            train = methyl_com[:n_train, :n_train]
            val = methyl_com[n_train:n_train+n_val, :n_train]
            test = methyl_com[n_train+n_val:, :n_train]
            sim_combined = np.vstack([train, val, test])
            similarity_matrices["mathylations"] = sim_combined
        
        if "cnvs" in typ:
            cnv_com = np.corrcoef(np.vstack([cnvs.loc[train_lines], 
                                             cnvs.loc[valid_validation_lines], 
                                             cnvs.loc[valid_test_lines]]), rowvar=True)
            train= cnv_com[:n_train, :n_train]
            val= cnv_com[n_train:n_train+n_val, :n_train]
            test= cnv_com[n_train+n_val:, :n_train]
            sim_combined = np.vstack([train, val, test])
            similarity_matrices["cnvs"] = sim_combined
            
        cell_dict = {}

        # 
        for cell in cell_line_index:
            cell_dict[cell] = {}
            for data_type in typ:
                sim_matrices = similarity_matrices[data_type]
                sim_tensor = torch.Tensor(sim_matrices)
                cell_idx = cell_line_index.get_loc(cell)
                cell_dict[cell][data_type] = sim_tensor[cell_idx]
                
        train_lines = train_lines
        validation_lines = valid_validation_lines
        test_lines = valid_test_lines

    # no change needed, query works fine with some missing
    train_data = data.query("SANGER_MODEL_ID in @train_lines")
    validation_data = data.query("SANGER_MODEL_ID in @validation_lines")
    test_data = data.query("SANGER_MODEL_ID in @test_lines")
    
    return (OmicsDataset_dict(cell_dict, drug_dict, train_data),
    OmicsDataset_dict(cell_dict, drug_dict, validation_data),
    OmicsDataset_dict(cell_dict, drug_dict, test_data))